Breaking down the files into the YOLO format and structure:

Folder structure:
* datasets
     * train
          * images
          * labels
     * test
          * images
          * labels
     * val
          * images
          * labels
     * (data.yaml)

label files target format:  
| detected class | bbox X-position | bbox Y-position | bbox Width | bbox Height |
|-|-|-|-|-|
| int | float (0<=X<1) | float (0<=Y<1) | float (0<=W<1) | float (0<=H<1) |


In [3]:
import json
import os

In [4]:
with open('datasets/val/images/_annotations.coco.json', 'r') as file:
    val_annotations = json.load(file)
val_annotations.keys()

dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])

In [5]:
val_annotations['categories']

[{'id': 0, 'name': 'spaces', 'supercategory': 'none'},
 {'id': 1, 'name': 'space-empty', 'supercategory': 'spaces'},
 {'id': 2, 'name': 'space-occupied', 'supercategory': 'spaces'}]

In [6]:
val_annotations['images'][0]

{'id': 0,
 'license': 1,
 'file_name': '2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a372345b23ed409.jpg',
 'height': 640,
 'width': 640,
 'date_captured': '2021-01-05T21:55:12+00:00'}

In [7]:
val_annotations['annotations'][0]

{'id': 0,
 'image_id': 0,
 'category_id': 2,
 'bbox': [322, 488, 80.5, 132.5],
 'area': 10666.25,
 'segmentation': [],
 'iscrowd': 0}

id, filename, height, width, bbox will be used to create the label files.
* id for merging in pandas
* filename.jpg -> filename.txt
* bbox [x,y,w,h] -> [X,Y,W,H]
* height, width -> float calculation

In [8]:
import pandas as pd
df_left = pd.DataFrame(val_annotations['images'])
df_left

,id,license,file_name,height,width,date_captured
0,0,1,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,2021-01-05T21:55:12+00:00
1,1,1,2013-03-20_07_50_02_jpg.rf.62ea5a91da3e88ec2d1...,640,640,2021-01-05T21:55:12+00:00
2,2,1,2012-09-12_12_16_56_jpg.rf.63035c9fd47dc1422cf...,640,640,2021-01-05T21:55:12+00:00
3,3,1,2012-09-17_14_24_11_jpg.rf.63304c3511762276636...,640,640,2021-01-05T21:55:12+00:00
4,4,1,2013-01-21_17_25_13_jpg.rf.6355eab55ae421d87bd...,640,640,2021-01-05T21:55:12+00:00
...,...,...,...,...,...,...
2478,2478,1,2012-12-21_11_00_06_jpg.rf.613e62f8983139306bb...,640,640,2021-01-05T21:55:12+00:00
2479,2479,1,2012-10-15_15_46_05_jpg.rf.614617a802b4e05697d...,640,640,2021-01-05T21:55:12+00:00
2480,2480,1,2012-12-20_16_25_11_jpg.rf.616628160446a16351a...,640,640,2021-01-05T21:55:12+00:00
2481,2481,1,2013-03-05_14_40_09_jpg.rf.61709962b9ec85832a5...,640,640,2021-01-05T21:55:12+00:00


In [9]:
df_right = pd.DataFrame(val_annotations['annotations'])
df_right

,id,image_id,category_id,bbox,area,segmentation,iscrowd
0,0,0,2,"[322, 488, 80.5, 132.5]",10666.25,[],0
1,1,0,1,"[351, 401, 74, 120]",8880.00,[],0
2,2,0,1,"[374, 355, 66, 89]",5874.00,[],0
3,3,0,2,"[386, 287, 64.5, 95]",6127.50,[],0
4,4,0,2,"[405, 251, 54, 76.5]",4131.00,[],0
...,...,...,...,...,...,...,...
143311,143311,2482,2,"[389, 38, 39.5, 36.5]",1441.75,[],0
143312,143312,2482,2,"[366, 11, 44.5, 36.5]",1624.25,[],0
143313,143313,2482,2,"[561, 75, 29.5, 61.5]",1814.25,[],0
143314,143314,2482,2,"[529, 50, 30.5, 50.5]",1540.25,[],0


In [10]:
df = pd.merge(df_left, df_right, how='left', left_on='id', right_on='image_id')
df

,id_x,license,file_name,height,width,date_captured,id_y,image_id,category_id,bbox,area,segmentation,iscrowd
0,0,1,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,2021-01-05T21:55:12+00:00,0.0,0.0,2.0,"[322, 488, 80.5, 132.5]",10666.25,[],0.0
1,0,1,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,2021-01-05T21:55:12+00:00,1.0,0.0,1.0,"[351, 401, 74, 120]",8880.00,[],0.0
2,0,1,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,2021-01-05T21:55:12+00:00,2.0,0.0,1.0,"[374, 355, 66, 89]",5874.00,[],0.0
3,0,1,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,2021-01-05T21:55:12+00:00,3.0,0.0,2.0,"[386, 287, 64.5, 95]",6127.50,[],0.0
4,0,1,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,2021-01-05T21:55:12+00:00,4.0,0.0,2.0,"[405, 251, 54, 76.5]",4131.00,[],0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
143370,2482,1,2013-03-14_11_10_58_jpg.rf.617de045146b2f636b6...,640,640,2021-01-05T21:55:12+00:00,143311.0,2482.0,2.0,"[389, 38, 39.5, 36.5]",1441.75,[],0.0
143371,2482,1,2013-03-14_11_10_58_jpg.rf.617de045146b2f636b6...,640,640,2021-01-05T21:55:12+00:00,143312.0,2482.0,2.0,"[366, 11, 44.5, 36.5]",1624.25,[],0.0
143372,2482,1,2013-03-14_11_10_58_jpg.rf.617de045146b2f636b6...,640,640,2021-01-05T21:55:12+00:00,143313.0,2482.0,2.0,"[561, 75, 29.5, 61.5]",1814.25,[],0.0
143373,2482,1,2013-03-14_11_10_58_jpg.rf.617de045146b2f636b6...,640,640,2021-01-05T21:55:12+00:00,143314.0,2482.0,2.0,"[529, 50, 30.5, 50.5]",1540.25,[],0.0


In [11]:
df = df.drop([
    'id_x', 
    'id_y',
    'license', 
    'date_captured', 
    'image_id', 
    'area', 
    'segmentation', 
    'iscrowd'], 
    axis=1)


In [12]:
df.isna().value_counts()

file_name  height  width  category_id  bbox 
False      False   False  False        False    143316
                          True         True         59
Name: count, dtype: int64

In [13]:
df = df.dropna()
df.isna().value_counts()

file_name  height  width  category_id  bbox 
False      False   False  False        False    143316
Name: count, dtype: int64

In [14]:
df

,file_name,height,width,category_id,bbox
0,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,2.0,"[322, 488, 80.5, 132.5]"
1,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,1.0,"[351, 401, 74, 120]"
2,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,1.0,"[374, 355, 66, 89]"
3,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,2.0,"[386, 287, 64.5, 95]"
4,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,2.0,"[405, 251, 54, 76.5]"
...,...,...,...,...,...
143370,2013-03-14_11_10_58_jpg.rf.617de045146b2f636b6...,640,640,2.0,"[389, 38, 39.5, 36.5]"
143371,2013-03-14_11_10_58_jpg.rf.617de045146b2f636b6...,640,640,2.0,"[366, 11, 44.5, 36.5]"
143372,2013-03-14_11_10_58_jpg.rf.617de045146b2f636b6...,640,640,2.0,"[561, 75, 29.5, 61.5]"
143373,2013-03-14_11_10_58_jpg.rf.617de045146b2f636b6...,640,640,2.0,"[529, 50, 30.5, 50.5]"


In [15]:
# Change category numbers to start from 0
print(df['category_id'].value_counts())
df['category_id'].loc[df['category_id'] == 1] = 0
df['category_id'].loc[df['category_id'] == 2] = 1
print(df['category_id'].value_counts())
df

category_id
1.0    73629
2.0    69687
Name: count, dtype: int64
category_id
0.0    73629
1.0    69687
Name: count, dtype: int64


/tmp/ipykernel_166829/1974903249.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['category_id'].loc[df['category_id'] == 1] = 0
/tmp/ipykernel_166829/1974903249.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pan

,file_name,height,width,category_id,bbox
0,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,1.0,"[322, 488, 80.5, 132.5]"
1,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,0.0,"[351, 401, 74, 120]"
2,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,0.0,"[374, 355, 66, 89]"
3,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,1.0,"[386, 287, 64.5, 95]"
4,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,1.0,"[405, 251, 54, 76.5]"
...,...,...,...,...,...
143370,2013-03-14_11_10_58_jpg.rf.617de045146b2f636b6...,640,640,1.0,"[389, 38, 39.5, 36.5]"
143371,2013-03-14_11_10_58_jpg.rf.617de045146b2f636b6...,640,640,1.0,"[366, 11, 44.5, 36.5]"
143372,2013-03-14_11_10_58_jpg.rf.617de045146b2f636b6...,640,640,1.0,"[561, 75, 29.5, 61.5]"
143373,2013-03-14_11_10_58_jpg.rf.617de045146b2f636b6...,640,640,1.0,"[529, 50, 30.5, 50.5]"


In [16]:
df['category_id'] = df['category_id'].convert_dtypes()
df

,file_name,height,width,category_id,bbox
0,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,1,"[322, 488, 80.5, 132.5]"
1,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,0,"[351, 401, 74, 120]"
2,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,0,"[374, 355, 66, 89]"
3,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,1,"[386, 287, 64.5, 95]"
4,2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a3...,640,640,1,"[405, 251, 54, 76.5]"
...,...,...,...,...,...
143370,2013-03-14_11_10_58_jpg.rf.617de045146b2f636b6...,640,640,1,"[389, 38, 39.5, 36.5]"
143371,2013-03-14_11_10_58_jpg.rf.617de045146b2f636b6...,640,640,1,"[366, 11, 44.5, 36.5]"
143372,2013-03-14_11_10_58_jpg.rf.617de045146b2f636b6...,640,640,1,"[561, 75, 29.5, 61.5]"
143373,2013-03-14_11_10_58_jpg.rf.617de045146b2f636b6...,640,640,1,"[529, 50, 30.5, 50.5]"


In [17]:
print(df.loc[1, 'file_name'])
df['file_name'] = df['file_name'].apply(func=lambda x: x[:len(x)-4]+'.txt' if str.endswith(x, '.jpg') else x)
print(df.loc[1, 'file_name'])

2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a372345b23ed409.jpg
2012-12-21_18_10_14_jpg.rf.62e89081eb0427cd3a372345b23ed409.txt


In [18]:
labels_path = 'datasets/val/labels'

for row, cols in df.iterrows():
    x,y,w,h = cols['bbox']
    X = (x+w / 2) / cols['width']
    Y = (y+h / 2) / cols['height']
    W = w / cols['width']
    H = h / cols['height']
    line = str(cols['category_id']) + ' ' + str(X) + ' ' + str(Y) + ' ' + str(W) + ' ' + str(H)+'\n'
    label_path = os.path.join(labels_path, cols['file_name'])
    with open(label_path, 'a') as file:
        file.writelines(line)